## **API WHEATHER**

 Ofrece una variedad de datos y funcionalidades para obtener información sobre el clima y las condiciones meteorológicas de cualquier lugar.

In [ ]:
import requests  # Biblioteca para realizar solicitudes HTTP
import matplotlib.pyplot as plt

# Función para obtener datos del clima desde WeatherAPI
def obtener_clima(ciudad, api_key_weather): # conecta con la API de Weather Api para opbtener los datos climáticos de una ciudad específica
    url_weather = f"http://api.weatherapi.com/v1/current.json?key={api_key_weather}&q={ciudad}" #Se construye URL dinámica, current.json para datos actuales, la clave de API (api_key_weather) y el parametro de busqueda (ciudad)
    try: #solicitamos datos a la API
        respuesta = requests.get(url_weather, timeout=10) #Hace una solicitud GET a la URL, limita el tiempo de espera 10seg
        respuesta.raise_for_status() # Lanza una excepción si la respuesta indica un error
        datos = respuesta.json() #Facilita el acceso convierte la respuesta JSON en un diccionario Python
        # Extraer datos relevantes
        clima = { #se construye diccionario clima con los datos relevantes como: Ciudad, país, temp °C, humedad%, velocidad viento km/h.
            "ciudad": datos.get("location", {}).get("name"), #usando get() se extraen valores especificos
            "pais": datos.get("location", {}).get("country"),
            "temperatura_c": datos.get("current", {}).get("temp_c"),
            "humedad": datos.get("current", {}).get("humidity"),
            "viento_kph": datos.get("current", {}).get("wind_kph"),
            "descripcion": datos.get("current", {}).get("condition", {}).get("text") #descripcion textual de las condiciones actuales
        }
        return clima
    except requests.exceptions.RequestException as e: #captura cualquier excepción generada durante la solicitud (error de conexion, tiempo de espera)
        print(f"Error al obtener el clima de {ciudad}: {e}")
        return None

# Ejemplo de uso
api_key_weather = "e1d44396e7d44430887215118241812"  # Reemplaza con tu clave de API
ciudad = "Lima" #se especifica la ciudad
clima = obtener_clima(ciudad, api_key_weather) #se llama la función y se guarda el resultado en clima.

if clima: #si clima no es None imprime los datos climáticos.
    print("Datos del clima:")
    print(f"Ciudad: {clima['ciudad']}, País: {clima['pais']}")
    print(f"Temperatura: {clima['temperatura_c']}°C")
    print(f"Humedad: {clima['humedad']}%")
    print(f"Viento: {clima['viento_kph']} km/h")
    print(f"Condiciones: {clima['descripcion']}")



Datos del clima:
Ciudad: Lima, País: Peru
Temperatura: 20.1°C
Humedad: 81%
Viento: 14.0 km/h
Condiciones: Partly Cloudy


## **API KEY IQAIR**
Esta API Ofrece información detallada sobre la calidad del aire (AQI) y contaminantes en diferentes ubicaciones del mundo.



In [ ]:
import requests

# Función para obtener datos de calidad del aire desde IQAir usando el nombre de una ciudad
def obtener_calidad_aire_ciudad(ciudad, estado, pais, api_key_iqair): #Obtiene datos de calidad del aire de IQAir para una ciudad específica.
    url_iqair = f"https://api.airvisual.com/v2/city?city={ciudad}&state={estado}&country={pais}&key={api_key_iqair}"
    try:
        respuesta = requests.get(url_iqair, timeout=10)  #solicita GET a la URL, limita el tiempo de espera 10seg
        respuesta.raise_for_status() # Lanza una excepción si la respuesta indica un error
        datos = respuesta.json() #Facilita el acceso convierte la respuesta JSON en un diccionario Python
        # Extraer datos relevantes
        calidad_aire = {
            "ciudad": datos.get("data", {}).get("city"),
            "estado": datos.get("data", {}).get("state"),
            "pais": datos.get("data", {}).get("country"),
            "aqi_us": datos.get("data", {}).get("current", {}).get("pollution", {}).get("aqius"),
            "principal_contaminante": datos.get("data", {}).get("current", {}).get("pollution", {}).get("mainus"),
            "temperatura_c": datos.get("data", {}).get("current", {}).get("weather", {}).get("tp"),
            "humedad": datos.get("data", {}).get("current", {}).get("weather", {}).get("hu"),
            "velocidad_viento_m/s": datos.get("data", {}).get("current", {}).get("weather", {}).get("ws")
        }
        return calidad_aire
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la calidad del aire para {ciudad}, {estado}, {pais}: {e}")
        return None

# Ejemplo de uso
api_key_iqair = "b79f3699-982d-4eb7-ac33-f15226399808"  # Reemplaza con tu clave de API
ciudad = "Lima"  # Ejemplo: ciudad
estado = "Lima"  # Ejemplo: estado
pais = "Peru"  # Ejemplo: país

calidad_aire = obtener_calidad_aire_ciudad(ciudad, estado, pais, api_key_iqair)

if calidad_aire:
    print("Datos de calidad del aire:")
    print(f"Ciudad: {calidad_aire['ciudad']}, Estado: {calidad_aire['estado']}, País: {calidad_aire['pais']}")
    print(f"AQI (US): {calidad_aire['aqi_us']}")
    print(f"Principal contaminante: {calidad_aire['principal_contaminante']}")
    print(f"Temperatura: {calidad_aire['temperatura_c']}°C")
    print(f"Humedad: {calidad_aire['humedad']}%")
    print(f"Velocidad del viento: {calidad_aire['velocidad_viento_m/s']} m/s")


Datos de calidad del aire:
Ciudad: Lima, Estado: Lima, País: Peru
AQI (US): 54
Principal contaminante: p2
Temperatura: 21°C
Humedad: 78%
Velocidad del viento: 6.69 m/s


## **Integración de APIS**

In [ ]:
import pandas as pd

# Función para comparar ciudades según su clima y calidad del aire
def comparar_ciudades(ciudades, api_key_iqair, api_key_weather):
    datos_aire = []  # Lista para almacenar datos de calidad del aire
    datos_clima = []  # Lista para almacenar datos del clima

    # Obtener datos de calidad del aire para cada ciudad
    for ciudad_info in ciudades:
        # Llama a la función obtener_calidad_aire para cada ciudad en la lista
        calidad_aire = obtener_calidad_aire(
            ciudad_info['ciudad'],  # Nombre de la ciudad
            ciudad_info['estado'],  # Nombre del estado
            ciudad_info['pais'],    # Nombre del país
            api_key_iqair           # Clave de acceso a la API IQAir
        )
        if calidad_aire:  # Verifica que los datos sean válidos
            datos_aire.append(calidad_aire)  # Agrega los datos a la lista

    # Obtener datos de clima para cada ciudad
    for ciudad_info in ciudades:
        # Llama a la función obtener_clima para cada ciudad en la lista
        clima = obtener_clima(
            ciudad_info['ciudad'],  # Nombre de la ciudad
            api_key_weather         # Clave de acceso a la API WeatherAPI
        )
        if clima:  # Verifica que los datos sean válidos
            datos_clima.append(clima)  # Agrega los datos a la lista

    # Crear un DataFrame de Pandas con los datos de calidad del aire
    df_aire = pd.DataFrame(datos_aire)  # Convierte la lista en un DataFrame

    # Crear un DataFrame de Pandas con los datos del clima
    df_clima = pd.DataFrame(datos_clima)  # Convierte la lista en un DataFrame

    # Combinar los DataFrames por el nombre de la ciudad
    comparacion_df = pd.merge(
        df_aire,      # Primer DataFrame: calidad del aire
        df_clima,     # Segundo DataFrame: datos del clima
        on="ciudad",  # Columna clave para combinar los DataFrames
        how="inner"   # Tipo de combinación: solo coincidencias en ambas tablas
    )

    return comparacion_df  # Devuelve el DataFrame combinado

# Definir las ciudades que deseas analizar
ciudades = [
    {"ciudad": "Piura", "estado": "Piura", "pais": "Peru"},         # Información de Piura
    {"ciudad": "Lima", "estado": "Lima", "pais": "Peru"},           # Información de Lima
    {"ciudad": "Arequipa", "estado": "Arequipa", "pais": "Peru"},   # Información de Arequipa
    {"ciudad": "Trujillo", "estado": "La Libertad", "pais": "Peru"} # Información de Trujillo
]

# Claves API
api_key_iqair = "88748538-fe54-4cf4-8ec8-6e0934fd13d0"  # Clave para IQAir
api_key_weather = "e1d44396e7d44430887215118241812"      # Clave para WeatherAPI

# Comparar las ciudades y almacenar el resultado en un DataFrame
comparacion = comparar_ciudades(ciudades, api_key_iqair, api_key_weather)

# Imprimir el DataFrame con los resultados
print(comparacion)


     ciudad       estado pais_x  aqi_us principal_contaminante  \
0     Piura        Piura   Peru      47                     p2   
1      Lima         Lima   Peru      54                     p2   
2  Arequipa     Arequipa   Peru      58                     p2   
3  Trujillo  La Libertad   Peru      37                     p2   

   temperatura_c_x  humedad_x  velocidad_viento_m/s pais_y  temperatura_c_y  \
0               22         66                  8.54   Peru             22.6   
1               21         78                  6.69   Peru             20.1   
2               16         63                  0.51   Peru             10.9   
3               18         82                  5.66   Peru             18.7   

   humedad_y  viento_kph    descripcion  
0         67        30.2  Partly Cloudy  
1         81        14.0  Partly Cloudy  
2         68         4.3          Clear  
3         86        10.1          Clear  
